# Universality of Single Qubit Gates

## Challenge Statement

In quantum computing, we have two fundamental structures: states and operators, which we can represent with vectors and matrices respectively. <br>

Vectors must have norm 1 (since they determine a sum of probabilities) and matrices must be unitary. <br>
  
When building a quantum computer, we would like it to be able to generate any  
U operator we need. However, we cannot physically implement every possible operator, so the goal is to create subsets of gates or operators that are able to generate all the others. <br>

To do so, you will use a variational method to find these parameters. Additionally, you must build an error function that tells us how close your matrix is to the input unitary U.

In [21]:
import json
import pennylane as qml
import pennylane.numpy as np

np.random.seed(1967)

def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
        
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    
    alpha, beta, gamma, phi = params

    # Put your code here #

    RZ_alpha = np.array([[np.exp(complex(0,-alpha/2)), 0], [0, complex(0, alpha/2)]])
    
    RX_beta = np.array([[np.cos(beta/2), complex(0,-np.sin(beta/2))], [np.sin(complex(0,-np.sin(beta/2))), np.cos(beta/2)]])
    
    RZ_gamma = np.array([[np.exp(complex(0,-gamma/2)), 0], [0, complex(0, gamma/2)]])
    
    the_matrix = np.multiply(np.exp(complex(0, phi)), np.matmul(RZ_gamma, np.matmul(RX_beta, RZ_alpha)))
    
    # Return the matrix
    return the_matrix


def error(U, params):
    """
    This function determines the similarity between your generated matrix and
    the target unitary.

    Args:
        - U (np.array): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)


    # Put your code here #

    error = float(sum(sum(np.absolute(matrix - U))))
    
    # Return the error
    return error


def train_parameters(U):
    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)

    return params


In [22]:
get_matrix([.2,.3,.4,.5])

tensor([[ 0.96906149+0.19643849j,  0.01427637+0.0044162j ],
        [ 0.0276309 +0.01168216j, -0.01735457-0.00948084j]], requires_grad=True)

In [23]:
test_input = [[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]
expected_output = [[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]
params = [.2,.3,.4,.5]
error(test_input, params)

2.3896650107139514

In [24]:
test_input = [[ 1,  0], [ 0, -1]]
expected_output = [[ 1,  0], [ 0, -1]]
params = [.2,.3,.4,.5]
error(test_input, params)

1.2264939041913188

In [ ]:
train_parameters(test_input)

In [25]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)


def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)


# These are the public test cases
test_cases = [
    ('[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'),
    ('[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
Runtime Error. complex() second argument must be a number, not 'ArrayBox'
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
Runtime Error. complex() second argument must be a number, not 'ArrayBox'
